<a href="https://colab.research.google.com/github/sanazgit/ReLIC_LSTM/blob/main/JAS_ReLIC_PPLUS_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision

In [2]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 13.2 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!sudo mkdir /content/drive1
!google-drive-ocamlfuse /content/drive1

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [7]:
# upload a pathon file with some useful functions
from google.colab import files
uploaded = files.upload()

Saving dataset.py to dataset.py
Saving mobilenetv2.pth to mobilenetv2.pth
Saving option_new.py to option_new.py
Saving util.py to util.py


In [8]:
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from tensorboardX import SummaryWriter
from sklearn.metrics import accuracy_score

In [9]:
!unzip /content/drive/MyDrive/Image_data.zip

Archive:  /content/drive/MyDrive/Image_data.zip
   creating: Image_data/
  inflating: Image_data/Aalto1.jpg   
  inflating: Image_data/Aalto2.jpg   
  inflating: Image_data/Aertsen1.jpg  
  inflating: Image_data/Aertsen3.jpg  
  inflating: Image_data/Aivazovsky 2.jpg  
  inflating: Image_data/Aivazovsky1.jpg  
  inflating: Image_data/Albani1.jpg  
  inflating: Image_data/Alcazar1.jpg  
  inflating: Image_data/Alcazar2.jpg  
  inflating: Image_data/Alcazar3.jpg  
  inflating: Image_data/Allori1.jpg  
  inflating: Image_data/Altdorfer 1.jpg  
  inflating: Image_data/Altdorfer1.jpg  
  inflating: Image_data/AmanJean1.jpg  
  inflating: Image_data/Amigoni1.jpg  
  inflating: Image_data/Arcimboldo1.jpg  
  inflating: Image_data/Arcimboldo2.jpg  
  inflating: Image_data/Arcimboldo3.jpg  
  inflating: Image_data/Arellano1.jpg  
  inflating: Image_data/Arentsz1.jpg  
  inflating: Image_data/Avercamp1.jpg  
  inflating: Image_data/Avercamp2.jpg  
  inflating: Image_data/Backer1.jpg  
  inflatin

In [10]:
from relic2_lstm_model import NIMA
from dataset import JASDataset
from util import AverageMeter
import option_new

opt = option_new.init()
opt.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def adjust_learning_rate(params, optimizer, epoch):
    """Sets the learning rate to the initial LR
       decayed by 10 every 30 epochs"""
    lr = params.init_lr * (0.1 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def create_data_part(opt):
    
    train_data = JASDataset(opt.path_to_train_csv,opt.path_to_imgs)
    test_data = JASDataset(opt.path_to_test_csv,opt.path_to_imgs)
    val_data = JASDataset(opt.path_to_val_csv,opt.path_to_imgs)

    train_loader = DataLoader(train_data,batch_size=opt.batch_size,num_workers= opt.num_workers,shuffle=True)
    test_loader = DataLoader(test_data,batch_size=opt.batch_size,num_workers= opt.num_workers,shuffle=False)
    val_loader = DataLoader(val_data,batch_size=opt.batch_size,num_workers= opt.num_workers,shuffle=False)
    

    return train_loader,test_loader,val_loader


def train(opt,model, loader, optimizer, criterion, writer=None, global_step=None, name=None):
    model.train()
    train_losses = AverageMeter()
    for idx, (x, y) in enumerate(tqdm(loader)):
        
        x = x.to(opt.device)
        y = y.to(opt.device)
        y_pred = model(x)
        y_pred = y_pred.squeeze(1)
        loss = criterion(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.update(loss.item(), x.size(0))
        

        if writer is not None:
            writer.add_scalar(f"{name}/train_loss.avg", train_losses.avg, global_step=global_step + idx)
    return train_losses.avg


def validate(opt,model, loader, criterion, writer=None, global_step=None, name=None):
    model.eval()
    validate_losses = AverageMeter()
    true_score = []
    pred_score = []
    
    for idx, (x, y) in enumerate(tqdm(loader)):
        
        x = x.to(opt.device)
        y = y.type(torch.FloatTensor)
        y = y.to(opt.device)

        y_pred = model(x)
        y_pred = y_pred.squeeze(1)

        loss = criterion(y_pred, y)
        validate_losses.update(loss.item(), x.size(0))
        

        if writer is not None:
            writer.add_scalar(f"{name}/val_loss.avg", validate_losses.avg, global_step=global_step + idx)

        for i in range(y.size(0)):
            true_score.append(y[i].data.cpu())
            pred_score.append(y_pred[i].data.cpu())

    lcc_mean = pearsonr(pred_score, true_score)
    srcc_mean = spearmanr(pred_score, true_score)
    print('lcc_mean', lcc_mean[0])
    print('srcc_mean', srcc_mean[0])

    true_score = np.array(true_score)
    true_score_lable = np.where(true_score <= 0.5, 0, 1)
    pred_score = np.array(pred_score)
    pred_score_lable = np.where(pred_score <= 0.5, 0, 1)
    acc = accuracy_score(true_score_lable, pred_score_lable)
    print(acc)

    return validate_losses.avg, acc, lcc_mean, srcc_mean


def start_train(opt):
    
    train_loader, val_loader, test_loader = create_data_part(opt)
    model = NIMA()
    optimizer = torch.optim.Adam(model.parameters(), lr=opt.init_lr)
    # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=opt.init_lr)
    criterion = nn.MSELoss()
    model = model.to(opt.device)
    criterion.to(opt.device)

    writer = SummaryWriter(log_dir=os.path.join(opt.experiment_dir_name, 'logs'))

    for e in range(opt.num_epoch):
        adjust_learning_rate(opt, optimizer, e)
    
        train_loss = train(opt,model=model, loader=train_loader, optimizer=optimizer, criterion=criterion,
                           writer=writer, global_step=len(train_loader) * e,
                           name=f"{opt.experiment_dir_name}_by_batch")
        val_loss,vacc,vlcc,vsrcc = validate(opt,model=model, loader=val_loader, criterion=criterion,
                            writer=writer, global_step=len(val_loader) * e,
                            name=f"{opt.experiment_dir_name}_by_batch")
        test_loss, tacc, tlcc, tsrcc = validate(opt, model=model, loader=test_loader, criterion=criterion,
                                               writer=writer, global_step=len(val_loader) * e,
                                               name=f"{opt.experiment_dir_name}_by_batch")
        model_name = f"epoch_{e}_.pth"
        torch.save(model.state_dict(), os.path.join(opt.experiment_dir_name, model_name))

        writer.add_scalars("epoch_loss", {'train': train_loss, 'val': val_loss, 'test': test_loss},
                           global_step=e)

        writer.add_scalars("lcc_srcc", {'val_lcc': vlcc[0], 'val_srcc': vsrcc[0],
                                        'test_lcc': tlcc[0], 'test_srcc': tsrcc[0]},
                           global_step=e)

        writer.add_scalars("acc",{'val_acc': vacc, 'test_acc': tacc}, global_step=e)

    writer.close()


def start_check_model(opt):
    _,test_loader, val_loader = create_data_part(opt)
    model = NIMA()
    model.eval()
    model.load_state_dict(torch.load(opt.path_to_model_weight))
    criterion = nn.MSELoss()

    model = model.to(opt.device)
    criterion.to(opt.device)

    test_loss, acc, lcc_mean, srcc_mean = validate(opt,model=model, loader=test_loader, criterion=criterion)

    print('loss:', test_loss, 'acc:', acc, 'lcc:', lcc_mean[0], 'srcc:', srcc_mean[0])


if __name__ =="__main__":

    ### train model
    start_train(opt)
    ### test model
    # start_check_model(opt)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 24/24 [00:40<00:00,  1.70s/it]


lcc_mean 0.5509386061403155
srcc_mean 0.49269462680316917
0.8315789473684211


100%|██████████| 16/16 [00:25<00:00,  1.62s/it]


lcc_mean 0.4586841363265102
srcc_mean 0.44895977377555446
0.7903225806451613


100%|██████████| 24/24 [00:40<00:00,  1.71s/it]


...

lcc_mean 0.47475532007224736
srcc_mean 0.47452098717299485
0.7903225806451613


100%|██████████| 24/24 [00:41<00:00,  1.71s/it]


lcc_mean 0.5905205418787184
srcc_mean 0.5449688971749149
0.8421052631578947


100%|██████████| 16/16 [00:25<00:00,  1.62s/it]

lcc_mean 0.48019397186655677
srcc_mean 0.47820139856947047
0.7741935483870968
